In [ ]:
import os
import json

from collections import defaultdict
from typing import Any, Optional

from utils import nested_dict

In [ ]:
def split_dataset(data):

    formatted_dataset = defaultdict(list)

    for data_point in data:
        
        question = data_point["instruction"]
        split_output = data_point["output"].split("\n")
        steps = len(split_output) - 1

        for i in range(steps):
            formatted_dataset[i+1].append(
                {
                    "source_question": question,
                    "source_steps": "\n".join(split_output[:i]),
                    "target_steps": "\n".join(split_output[i:i+1]),
                    "target_result": f"{split_output[-1].replace("#### ", "")}" if i == steps-1 else ""
                }
            )

    return formatted_dataset

In [ ]:
def create_dirs(formatted_data_dir: str):
    splits = ["test", "train"]
    types = ["normal", "socratic"]
    for split in splits:
        for type_ in types:
            path = os.path.join(formatted_data_dir, split, type_)
            if not os.path.exists(path):
                os.makedirs(path)

In [ ]:
filtered_data_dir = "data/filtered/"
data_paths = ["test.json", "train.json", "test_socratic.json", "train_socratic.json"]
create_dirs(formatted_data_dir="data/decomposed/")

for data_path in data_paths:

    with open(os.path.join(filtered_data_dir, data_path), "r") as f:
        data = json.load(f)

    formatted_dataset = split_dataset(data)
    split = "test" if "test" in data_path else "train"
    type_ = "socratic" if "socratic" in data_path else "normal"
    for i in range(1, 9):
        with open(f"data/decomposed/{split}/{type_}/{i}.json", "w") as f:
            json.dump(formatted_dataset[i], f, indent=4, ensure_ascii=False)
